In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

In [7]:
class CustomDataset(Dataset):
    def __init__(self,filepath,word2idx,tag2idx):
        self.word2idx = word2idx
        self.tag2idx = tag2idx
        self.sentences , self.ner_tags = self.load_data(filepath)

    def load_data(self,filepath):
        sentences, ner_tags = [],[]
        sentence, ner_tag = [],[]
        with open(filepath,"r",encoding = "utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    word,_,ner = line.split("\t")
                    sentence.append(word)
                    ner_tag.append(ner)
                else:
                    if sentence:
                        sentences.append(sentence)
                        ner_tags.append(ner_tag)
                        sentence, ner_tag = [], []  # rest after every sentence
            return sentences,ner_tags
    def __len__(self):
      return len(self.sentences)

    def __getitem__(self, idx):
        word_ids = [self.word2idx.get(w, self.word2idx[""]) for w in self.sentences[idx]]
        tag_ids = [self.tag2idx.get(t, self.tag2idx[""]) for t in self.ner_tags[idx]]

        return word_ids, tag_ids

def collate_fn(batch):
  sentences,tag = zip(*batch)
  max_len = max(len(s) for s in sentences)
  padded_sentences = [  s+ [0] * (max_len-len(s)) for s in sentences]
  padded_tag = [ t + [0] *(max_len-len(t)) for t in tag]

  return torch.tensor(padded_sentences),torch.tensor(padded_tag)

##### Load Dataset

In [8]:
train_path = "/content/drive/MyDrive/MyanmarNER/train_v5.conll"
val_path = "/content/drive/MyDrive/MyanmarNER/val_v5.conll"
test_path = "/content/drive/MyDrive/MyanmarNER/test_v5.conll"

word2idx = {"": 0, "": 1}
tag2idx = {"": 0, "": 1}
train_data = CustomDataset(train_path,word2idx, tag2idx)
val_data = CustomDataset(val_path,word2idx, tag2idx)
test_data = CustomDataset(test_path,word2idx, tag2idx)

train_load = DataLoader(train_data, batch_size=32, collate_fn=collate_fn)
val_load = DataLoader(val_data, batch_size=32, collate_fn=collate_fn)
test_load = DataLoader(test_data, batch_size=32, collate_fn=collate_fn)

In [9]:

for batch in train_load:
    sentences, tags = batch
    print("Sentences shape:", sentences.shape)
    print("Tags shape:", tags.shape)
    break


Sentences shape: torch.Size([32, 41])
Tags shape: torch.Size([32, 41])


##### Pretrained FastText Embedding

In [ ]:
# Force reinstall compatible versions
# !pip install numpy==1.24.3 --force-reinstall
# !pip install gensim --force-reinstall

%pip install numpy==1.24.3 --force-reinstall
%pip install gensim --force-reinstall

In [ ]:
from gensim.models import KeyedVectors

# Load FastText Burmese .vec file (ensure it's already uploaded or on your drive)
fasttext_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/MyanmarNER/cc.my.300.vec', binary=False)
# https://fasttext.cc/docs/en/crawl-vectors.html choose Burmese choose text .vec file

In [11]:
from collections import defaultdict
import numpy as np

def build_vocab(dataset):
    word_set = set()
    for sentence in dataset.sentences:
        for word in sentence:
            word_set.add(word)
    return word_set

# Create vocab from dataset (not DataLoader!)
vocab = build_vocab(train_data)

word2idx = {"<PAD>": 0, "<UNK>": 1}
embedding_dim = 300
embedding_matrix = []

# Initialize PAD and UNK
embedding_matrix.append(np.zeros(embedding_dim))  # <PAD>
embedding_matrix.append(np.random.uniform(-0.25, 0.25, embedding_dim))  # <UNK>

for word in vocab:
    word2idx[word] = len(word2idx)
    if word in fasttext_model:
        embedding_matrix.append(fasttext_model[word])
    else:
        embedding_matrix.append(np.random.uniform(-0.25, 0.25, embedding_dim))

embedding_matrix = np.array(embedding_matrix)


In [12]:
train_data = CustomDataset("/content/drive/MyDrive/MyanmarNER/train_v5.conll", word2idx, tag2idx)
val_data = CustomDataset("/content/drive/MyDrive/MyanmarNER/val_v5.conll", word2idx, tag2idx)
test_data = CustomDataset("/content/drive/MyDrive/MyanmarNER/test_v5.conll", word2idx, tag2idx)


In [14]:
import torch.nn as nn

class NERModel(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, tagset_size):
        super(NERModel, self).__init__()

        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix), freeze=False
        )
        self.lstm = nn.LSTM(input_size=embedding_matrix.shape[1],
                            hidden_size=hidden_dim,
                            num_layers=1,
                            bidirectional=True,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, tagset_size)  # because it's bidirectional

    def forward(self, input_ids):
        embeds = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embeds)
        output = self.fc(lstm_out)
        return output



In [15]:
model = NERModel(embedding_matrix=embedding_matrix, hidden_dim=128, tagset_size=len(tag2idx))
